In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

df = pd.read_csv("train.csv")
df.columns = df.columns.str.replace('[?>% ]','')

#temporarily fill na
df.education=df.education.fillna("Bachelor's")
df.previous_year_rating=df.previous_year_rating.fillna(int(3))
dfnew=pd.get_dummies(df)

y = dfnew.is_promoted
x_train, x_test, y_train, y_test = train_test_split(dfnew.drop(['is_promoted'],axis=1),y,test_size=0.35,random_state=42)
train_idx = x_train.employee_id
test_idx = x_test.employee_id
x_train = x_train.drop(['employee_id'],axis=1)
x_test = x_test.drop(['employee_id'],axis=1)
pred = pd.DataFrame()
for i in range(5):
    #random.seed(random.randint(1,100))
    clf = XGBRegressor()
    model = clf.fit(X=x_train,y=y_train)
    yhat = model.predict(x_test)
    pred = pd.concat([pred,pd.DataFrame(yhat)],axis=1)

pred['prediction'] = pred.apply(lambda x: x.mean() ,axis=1)

pred = pred.prediction
cutoff = 0.24
pred[pred<=cutoff] = 0
pred[pred>cutoff] = 1
## test
sub = pd.read_csv('test.csv')
sub.columns = sub.columns.str.replace('[?>% ]','')
#temporarily fill na
sub.education=sub.education.fillna("Bachelor's")
sub.previous_year_rating = sub.previous_year_rating.fillna(int(3))
subnew=pd.get_dummies(sub)
submission = model.predict(subnew.drop(['employee_id'],axis=1))
submission[submission<=0.24] = 0
submission[submission>0.24] = 1
submission = pd.concat([subnew.employee_id,pd.DataFrame(submission)],axis=1)
submission.columns = ['employee_id','is_promoted']
submission.to_csv('wns_final_submission.csv',index=False)
